In [ ]:
%load_ext autoreload
%autoreload 2

# mount drive
from google.colab import drive
drive.mount('/content/drive')

# cd into project directory
%cd /content/drive/My\ Drive/Georgia_Tech/Spring_2021/sbic_stereotypes/src

In [ ]:
!pip install transformers
!pip install datasets

import torch
import pandas as pd
import numpy as np

# Useful constants
CLASSIFIER_MODEL_NAME = 'bert-base-uncased'
CLASSIFIERS = ['./classification/model/whoTarget/checkpoint-1280/']

#CLASSIFIERS = ['./classification/model/offensiveYN/checkpoint-898/',
#               './classification/model/intentYN/checkpoint-898/',
#               './classification/model/sexYN/checkpoint-898/',
#               './classification/model/whoTarget/checkpoint-1280/']

SEQ2SEQ_MODEL_NAME = 'facebook/bart-base'
SEQ2SEQ_MODEL = 'model/checkpoint-10782/'
DATA_DIR = '../data/'

In [ ]:
from seq2seq_utils import *
from torch import nn, torch
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BartTokenizer, BartForConditionalGeneration

df = pd.read_csv(DATA_DIR + 'SBIC.v2.dev.csv')
clean_post(df)
df = clean_target(df)
df.to_csv('data/clean_dev_df.csv')

dataset = Dataset.from_pandas(df)
seq2seq_tok, classifier_tok, tokenized = get_tokenized_data(
    dataset,
    SEQ2SEQ_MODEL_NAME,
    CLASSIFIER_MODEL_NAME
)

print(tokenized)

In [ ]:
from transformers import BartForConditionalGeneration
from seq2seq import BartForConditionalGenerationJoinModel
import torch

### MODEL START CODE ###

### MODEL END CODE ###
model = BartForConditionalGenerationJoinModel.from_pretrained(
            SEQ2SEQ_MODEL_NAME,
            classifiers=CLASSIFIERS,
        )
model.eval();

In [ ]:
i = 345
j = 350

#input_ids = torch.tensor([tokenized['input_ids'][i]])
#attention_mask = torch.tensor([tokenized['attention_mask'][i]])
#classifier_inputs = torch.tensor([tokenized['classifier_inputs'][i]])
#classifier_attention = torch.tensor([tokenized['classifier_attention'][i]])

input_ids = torch.tensor(tokenized['input_ids'][i:j])
attention_mask = torch.tensor(tokenized['attention_mask'][i:j])
classifier_inputs = torch.tensor(tokenized['classifier_inputs'][i:j])
classifier_attention = torch.tensor(tokenized['classifier_attention'][i:j])
num_beams = 10

encoder_outputs = model.get_encoder()(
    input_ids,
    attention_mask,
    return_dict=True,
)

kwargs = {
    'attention_mask':attention_mask,
    'classifier_inputs': classifier_inputs,
    'classifier_attention': classifier_attention,
    'beam_search': num_beams > 1,
    'encoder_outputs': encoder_outputs,
}

#for i in range(5):
#  model(input_ids=input_ids, attention_mask=attention_mask, classifier_inputs=classifier_inputs, classifier_attention=classifier_attention);
outputs = model.generate(input_ids, num_beams=num_beams, **kwargs)
seq2seq_tok.decode(outputs[0])